## Seminar Assignment 1
Inteligentni sistemi, 20. 11. 2022   
Klemen Bogataj in Aljaž Rakovec

Seminarsko nalogo sva delala v Pythonu, za genetski algoritem pa sva uporabila knjižnico PyGAD (https://github.com/ahmedfgad/GeneticAlgorithmPython).

#### Reprezentacija
Labirint preberemo v NumPy matriko, kjer številke predstavljajo različna polja:
- \# -> 0
- . -> 1
- S -> 2
- E -> 3
- T -> 4

Vse ostale parametre določimo ob inicializaciji genetskega algoritma.

```Python
char2num = {"#": 0, ".": 1, "S": 2, "E": 3, "T": 4}
def convert(mazeChars):
    return np.array([[char2num[char] for char in line] for line in mazeChars], dtype=np.int32)
```

### Fitness funkcije
- Fitness funkcija klemen1
Iz reštve vzamemo samo prvih toliko potez, dokler ne pridemo do zidu ali cilja. Če smo dosegli cilj, je vrednost 1000 - dolžina poti, torej kaznujemo daljše poti, če cilja še nismo dosegli, pa je vrednost število obiskanih polj - dolžina poti / 1000, torej nagradimo rešitve, ki prepodujejo čimvečji del labirinta in jim to uspe s čim manj potezami.
Težava je, da ne upoštevamo poti po prvem zadetem zidu, torej ne moremo oceniti uspešnosti sprememb od tam naprej.
- Fitness funkcija klemen2
Tukaj za začetno populacijo generiramo rešitve, ki vse pridejo do cilja, ampak grejo čez zidove. Vzamemo poteze od začetka do prvega prispetja na cilj. Za ta del poti izračunamo število prepotovanih različnih veljavnih polj, število potez, ki končajo izven mej labirinta in število zadetih zidov. Če smo do cilja prišli po samih veljavnih poljih, je vrednost 1000000 - dolžina poti, sicer pa najbolj kaznujemo poteze izven labirinta, nato zadete zidove, nagradimo pa poti, ki obiščejo čimveč različnih veljavnih polj.
- Fitness funkcija aljaz

Končna fitness funkcija:<br>
Ker zdaj vse rešitve hodijo samo po veljavnih poljih, se ni traba ukvarjati z zidovi. Dokler še nismo na cilju nagradimo daljše poti ter obilno nagradimo vsak najden zaklad, ko pa prispemo na cilj, pa poleg nagrajevanja zakladov še obilneje nagradimo prispetje na cilj, dolžino poti pa tokrat kaznujemo.

```Python
# insert
```

### Crossover funkcija

- Uniform
- None
- crossover aljaz

Končna Crossover funkcija:<br>
V dveh rešitvah poiščemo skupne odločitvene točke, torej tiste, ki so vsaj enkrat obiskane pri obeh starših. Naključno izberemo eno izmed njih, in ker se lahko v rešitvi pojavi večkrat, še nakljušno izberemo eno izmed pojavitev. Potomce generiramo tako, da vzamemo del poti od začetka do izbrane pojavitve od enega starša, drugi del od te točke naprej pa od drugega. tudi ostala dela združimo, tako da imamo dva nova potomca. Ker lahko pride do drugačne dolžine kromosoma, daljšega na koncu odrežemo na želeno dolžino, krajšemu pa na koncu dodamo nove poteze tako, kot smo jih generirali za začetno populacijo. Tako dosežemo, da še vedno nobena pot nikoli ne gre čez zid.

```Python
# insert
```

### Mutation funkcija

- Mutacija aljaz
- Mutacija klemen
Izberemo naljučno mesto od začetka do prvega prispetja na cilj in tam izvedemo mutacijo odvisno od naslednjih dveh potez:
    - Če sta potezi v eno smer in nazaj, izbrišemo te dve potezi , vse naslednje premeknemo za dve mesti v levo in na koncu dodamo dve naključni novi.
    - Če sta potezi pravokotni, ju zamenjamo, torej na nek način obrnemo vogal, ki smo ga našli.
    - Če sta potezi obe v isto smer, gremo po eni strani okrog vmesnega polja. Torej pred njiju vrinemo eno pravokotno potezo, za njiju pa pravokotno potezo ravno v drugo stran. Vse za tem zamaknemo za dve mesti naprej, pri tem se zadnji dve izbrišeta. LL -> ULLD / DLLU
Na ta način dosežemo, da poti pred in po mutaciji gresta po istem delu labirinta kot prej, in s tem preprečimo možnost, da pot ne bi več prišla do cilja.

Končna mutation funkcija:

```Python
# insert
```

### Inicializacija genteskega algoritma

Začetna inicializacija začetne populacije:<br>
Generiramo poteze, ki ne grejo nujno po veljavnih poljih, vendar približno na polovici pridejo do cilja. To dosežemo tako, da izračunamo oddaljenost starta in cilja po obeh dimenzijah, zgeneriramo ustrezno večje ali manjše število potez gor, dol ter levo, desno, nato pa te poteze naključno premešamo. Preostali del poti zapolnimo z naključnimi potezami. Cilj na sredini omogoča, da se pot do tja skrajšuje ali podaljšuje, ne da bi cilj izpadel iz poti.


Končna inicializacija začetne populacije:<br>
V večini točk je nesmiselno iti v smer zidu ali nazaj, od koder smo prišli, zato najprej v labirintu poiščemo "točke odločitve", to so start, cilj, zakladi in veljavna polja, ki imajo vsaj tri sosednja veljavna polja. To pomeni, da če gremo od ene izmed teh točk v neko smer, nadaljujemo po poti vse do naslednje take točke, ne da bi se vmes obračali ali zaletavali v zid. Zato zdaj nima več smisla v populacijo pisati vseh potez, ampak samo poteze na teh točkah, saj so povezave med njimi jasne. Začetne rešitve torej generiramo tako, da začnemo na cilju, izberemo eno imed veljavnih potez, pogledamo do katere odločitvene točke bomo prišli po tej poti, nato pa pri te in vseh nasledjih odločitvenih točkah spet naključno izberemo eno izmed možnih veljavnih potez. Tako so vse rešitve zapisane v veliko krajši obliki in vse se po labitintu sprehajajo samo po veljavnih poljih.

```Python
# insert 
```

Kreiranje instance genetskega algoritma:

```Python
ga_instance = pygad.GA(num_generations=2000,
                       num_parents_mating=10,
                       initial_population=population,
                       gene_space=[0, 1, 2, 3],
                       keep_elitism=5,
                       stop_criteria="saturate_200",
                       crossover_type=crossover_func,
                       mutation_probability=1,
                       mutation_type=mutation_func,
                       fitness_func=fitness_func,
                       on_generation=on_generation,
                       on_stop=on_stop)
```

Zagon genetskega algoritma:

```Python
ga_instance.run()

```